# Autoflow

Autoflow is a RAG framework supported:

- Vector Search Based RAG
- Knowledge Graph Based RAG (aka. GraphRAG)
- Knowledge Base and Document Management

## Installation

In [1]:
%pip install -q autoflow-ai==0.0.2.dev3 ipywidgets

Note: you may need to restart the kernel to use updated packages.


## Prerequisites

- Go [tidbcloud.com](https://tidbcloud.com/) or using [tiup playground](https://docs.pingcap.com/tidb/stable/tiup-playground/) to create a free TiDB database cluster.
- Go [OpenAI platform](https://platform.openai.com/api-keys) to create your API key.

#### For Jupyter Notebook

Configuration can be provided through environment variables, or using `.env`:

In [2]:
%%bash

# Check if the .env file is existing.
if [ -f .env ]; then
    exit 0
fi

# Create .env file with your configuration.
cat > .env <<EOF
TIDB_HOST=localhost
TIDB_PORT=4000
TIDB_USERNAME=root
TIDB_PASSWORD=
TIDB_DATABASE=test
OPENAI_API_KEY='your_openai_api_key'
EOF

In [3]:
import os
import dotenv

dotenv.load_dotenv()

True

In [4]:
from pandas import DataFrame
from pandas import set_option

set_option("display.max_colwidth", None)

## Quickstart

### Init Autoflow

In [5]:
from autoflow import Autoflow
from autoflow.configs.db import DatabaseConfig
from autoflow.configs.main import Config

af = Autoflow.from_config(
    config=Config(
        db=DatabaseConfig(
            host=os.getenv("TIDB_HOST"),
            port=int(os.getenv("TIDB_PORT")),
            username=os.getenv("TIDB_USERNAME"),
            password=os.getenv("TIDB_PASSWORD"),
            database=os.getenv("TIDB_DATABASE"),
            enable_ssl=False,
        )
    )
)

### Create knowledge base

In [6]:
from autoflow.configs.knowledge_base import IndexMethod
from autoflow.models.llms import LLM
from autoflow.models.embedding_models import EmbeddingModel

llm = LLM("gpt-4o-mini")
embed_model = EmbeddingModel("text-embedding-3-small")

kb = af.create_knowledge_base(
    namespace="quickstart",
    name="New KB",
    description="This is a knowledge base for testing",
    index_methods=[IndexMethod.VECTOR_SEARCH, IndexMethod.KNOWLEDGE_GRAPH],
    llm=llm,
    embedding_model=embed_model,
)
kb.model_dump_json()

'{"namespace":"quickstart","name":"New KB","description":"This is a knowledge base for testing","index_methods":["VECTOR_SEARCH","KNOWLEDGE_GRAPH"],"class_name":"KnowledgeBase"}'

In [7]:
# Reset all the data of knowledge base.
kb.reset()

### Custom Chunker

In [8]:
from autoflow.chunkers.text import TextChunker
from autoflow.configs.chunkers.text import TextChunkerConfig

text_chunker = TextChunker(config=TextChunkerConfig(chunk_size=256, chunk_overlap=20))

### Import documents from files

In [9]:
docs = kb.add("./fixtures/tidb-overview.md", chunker=text_chunker)

DataFrame(
    [(c.id, c.text) for c in docs[0].chunks],
    columns=["id", "text"],
)

,id,text
0,019637a6-fc49-78b4-8e55-a2ed1a79f00e,"---\ntitle: What is TiDB Self-Managed\nsummary: Learn about the key features and usage scenarios of TiDB.\naliases: ['/docs/dev/key-features/','/tidb/dev/key-features','/docs/dev/overview/']\n---\n\n# What is TiDB Self-Managed\n\n<!-- Localization note for TiDB:\n\n- English: use distributed SQL, and start to emphasize HTAP\n- Chinese: can keep ""NewSQL"" and emphasize one-stop real-time HTAP (""一栈式实时 HTAP"")\n- Japanese: use NewSQL because it is well-recognized\n\n-->\n\n[TiDB](https://github.com/pingcap/tidb) (/'taɪdiːbi:/, ""Ti"" stands for Titanium) is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads. It is MySQL compatible and features horizontal scalability, strong consistency, and high availability. The goal of TiDB is to provide users with a one-stop database solution that covers OLTP (Online Transactional Processing), OLAP (Online Analytical Processing), and HTAP services. TiDB is suitable for various use cases that require high availability and strong consistency with large-scale data."
1,019637a6-fc49-78f9-b6e8-4a43a834d5fb,"TiDB is suitable for various use cases that require high availability and strong consistency with large-scale data.\n\nTiDB Self-Managed is a product option of TiDB, where users or organizations can deploy and manage TiDB on their own infrastructure with complete flexibility. With TiDB Self-Managed, you can enjoy the power of open source, distributed SQL while retaining full control over your environment.\n\nThe following video introduces key features of TiDB.\n\n<iframe width=""600"" height=""450"" src=""https://www.youtube.com/embed/aWBNNPm21zg?enablejsapi=1"" title=""Why TiDB?"" frameborder=""0"" allow=""accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture"" allowfullscreen></iframe>\n\n## Key features\n\n- **Easy horizontal scaling**\n\n The TiDB architecture design separates computing from storage, letting you scale out or scale in the computing or storage capacity online as needed. The scaling process is transparent to application operations and maintenance staff.\n\n- **Financial-grade high availability**\n\n Data is stored in multiple replicas, and the Multi-Raft protocol is used to obtain the transaction log."
2,019637a6-fc49-790e-959d-2656e59449ab,"A transaction can only be committed when data has been successfully written into the majority of replicas. This guarantees strong consistency and availability when a minority of replicas go down. You can configure the geographic location and number of replicas as needed to meet different disaster tolerance levels.\n\n- **Real-time HTAP**\n\n TiDB provides two storage engines: [TiKV](/tikv-overview.md), a row-based storage engine, and [TiFlash](/tiflash/tiflash-overview.md), a columnar storage engine. \n\n TiFlash uses the Multi-Raft Learner protocol to replicate data from TiKV in real time, ensuring consistent data between the TiKV row-based storage engine and the TiFlash columnar storage engine. TiKV and TiFlash can be deployed on different machines as needed to solve the problem of HTAP resource isolation.\n\n- **Cloud-native distributed database**\n\n TiDB is a distributed database designed for the cloud, providing flexible scalability, reliability, and security on the cloud platform. Users can elastically scale TiDB to meet the requirements of their changing workloads."
3,019637a6-fc49-791e-ae05-2874c7aa5975,"Users can elastically scale TiDB to meet the requirements of their changing workloads. In TiDB, each piece of data has at least 3 replicas, which can be scheduled in different cloud availability zones to tolerate the outage of a whole data center. [TiDB Operator](https://docs.pingcap.com/tidb-in-kubernetes/stable/tidb-operator-overview) helps manage TiDB on Kubernetes and automates tasks related to operating the TiDB cluster, making TiDB easier to deploy on any cloud that provides 

### Search Documents

In [10]:
result = kb.search_documents(
    query="What is TiDB?",
    top_k=3,
)

DataFrame(
    [(c.text, c.score) for c in result.chunks],
    columns=["text", "score"],
)

,text,score
0,"---\ntitle: What is TiDB Self-Managed\nsummary: Learn about the key features and usage scenarios of TiDB.\naliases: ['/docs/dev/key-features/','/tidb/dev/key-features','/docs/dev/overview/']\n---\n\n# What is TiDB Self-Managed\n\n<!-- Localization note for TiDB:\n\n- English: use distributed SQL, and start to emphasize HTAP\n- Chinese: can keep ""NewSQL"" and emphasize one-stop real-time HTAP (""一栈式实时 HTAP"")\n- Japanese: use NewSQL because it is well-recognized\n\n-->\n\n[TiDB](https://github.com/pingcap/tidb) (/'taɪdiːbi:/, ""Ti"" stands for Titanium) is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads. It is MySQL compatible and features horizontal scalability, strong consistency, and high availability. The goal of TiDB is to provide users with a one-stop database solution that covers OLTP (Online Transactional Processing), OLAP (Online Analytical Processing), and HTAP services. TiDB is suitable for various use cases that require high availability and strong consistency with large-scale data.",0.726047
1,"TiDB is suitable for various use cases that require high availability and strong consistency with large-scale data.\n\nTiDB Self-Managed is a product option of TiDB, where users or organizations can deploy and manage TiDB on their own infrastructure with complete flexibility. With TiDB Self-Managed, you can enjoy the power of open source, distributed SQL while retaining full control over your environment.\n\nThe following video introduces key features of TiDB.\n\n<iframe width=""600"" height=""450"" src=""https://www.youtube.com/embed/aWBNNPm21zg?enablejsapi=1"" title=""Why TiDB?"" frameborder=""0"" allow=""accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture"" allowfullscreen></iframe>\n\n## Key features\n\n- **Easy horizontal scaling**\n\n The TiDB architecture design separates computing from storage, letting you scale out or scale in the computing or storage capacity online as needed. The scaling process is transparent to application operations and maintenance staff.\n\n- **Financial-grade high availability**\n\n Data is stored in multiple replicas, and the Multi-Raft protocol is used to obtain the transaction log.",0.669803
2,"Users can elastically scale TiDB to meet the requirements of their changing workloads. In TiDB, each piece of data has at least 3 replicas, which can be scheduled in different cloud availability zones to tolerate the outage of a whole data center. [TiDB Operator](https://docs.pingcap.com/tidb-in-kubernetes/stable/tidb-operator-overview) helps manage TiDB on Kubernetes and automates tasks related to operating the TiDB cluster, making TiDB easier to deploy on any cloud that provides managed Kubernetes. [TiDB Cloud](https://pingcap.com/tidb-cloud/), the fully-managed TiDB service, is the easiest, most economical, and most resilient way to unlock the full power of [TiDB in the cloud](https://docs.pingcap.com/tidbcloud/), allowing you to deploy and run TiDB clusters with just a few clicks.\n\n- **Compatible with the MySQL protocol and MySQL ecosystem**\n\n TiDB is compatible with the MySQL protocol, common features of MySQL, and the MySQL ecosystem. To migrate applications to TiDB, you do not need to change a single line of code in many cases, or only need to modify a small amount of code.",0.656657


### Search Knowledge Graph

In [11]:
kg = kb.search_knowledge_graph(
    query="What is TiDB?",
)

# Notice: socre is the result of a weighted formula

DataFrame(
    [
        (r.source_entity.name, r.description, r.target_entity.name, r.score)
        for r in kg.relationships
    ],
    columns=["source_entity", "relation", "target_entity", "score"],
)

,source_entity,relation,target_entity,score
0,TiDB,TiDB provides TiKV as a row-based storage engine.,TiKV,6.339184
1,TiDB,TiDB Storage is a component of TiDB that manages data storage and retrieval.,TiDB Storage,5.665690
2,TiDB,TiDB has key features that include easy horizontal scaling and financial-grade high availability.,Key features of TiDB,5.604980
3,TiDB,"TiDB also provides OLAP services, enabling fast and interactive access to data for analysis.",OLAP (Online Analytical Processing),5.123806
4,TiDB,"TiDB is designed for high availability, ensuring operational continuity even during failures or maintenance.",High Availability,5.117462
5,TiDB,"TiDB provides strong consistency, ensuring that the most recent write is visible to all subsequent reads.",Strong Consistency,5.074885
6,TiDB,"TiDB is MySQL compatible, enabling users to utilize existing MySQL applications and tools.",MySQL Compatibility,5.029335
7,TiDB,"TiDB features horizontal scalability, allowing it to expand capacity by adding more machines or nodes.",Horizontal Scalability,4.942130
8,TiDB,TiDB provides TiFlash as a columnar storage engine.,TiFlash,4.787550
9,TiDB,TiDB Architecture is a related concept that describes the structural design of the TiDB database.,TiDB Architecture,4.772436


### Ask question

In [13]:
res = kb.ask("What is TiDB?")
str(res)

AttributeError: 'KnowledgeBase' object has no attribute 'ask'

### Reset the KnowledgeBase

In [12]:
# kb.reset()